In [1]:
# ============================================================
# 01 - Importación de librerías
# ============================================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os, time, sys, glob, shutil, json


# ============================================================
# 02 - Cargar configuración desde JSON (credenciales + URLs)
# ============================================================
CONFIG_PATH = "ConfigWBEURO.json"

if not os.path.isfile(CONFIG_PATH):
    raise FileNotFoundError(f"[ERROR] No se encontró el archivo {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

USER = config["USER"]
PWD  = config["PWD"]
LOGIN_URL = config["LOGIN_URL"]
CSPS_URL  = config["CSPS_URL"]


# ============================================================
# 03 - Configuración de rutas de archivos
# ============================================================
PARTS_DIR  = os.path.abspath(r"C:\partes_txt")       # Carpeta con .txt de entrada
OUTPUT_DIR = os.path.abspath(r"C:\descargas\EURO")  # Carpeta de salida para los .xls


# ============================================================
# 04 - Configuración del navegador (Chrome)
# ============================================================
options = Options()
options.add_argument("--start-maximized")
prefs = {"download.default_directory": OUTPUT_DIR}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)


# ============================================================
# 05 - Helpers (funciones auxiliares reutilizables)
# ============================================================
def safe_find(by, selector, timeout=30):
    """Encuentra un elemento y espera hasta que esté presente en el DOM."""
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((by, selector))
    )

def safe_click(by, selector, timeout=30):
    """Hace clic en un elemento asegurándose de que es clickable."""
    el = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by, selector))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    el.click()
    return el

def switch_to_frame(name, timeout=30):
    """Cambia al frame especificado."""
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.NAME, name))
    )

def back_to_default():
    """Regresa al contenido principal fuera de los frames."""
    driver.switch_to.default_content()

def get_latest_download(path=OUTPUT_DIR, ext="xls"):
    """Obtiene el archivo más reciente descargado en OUTPUT_DIR."""
    files = glob.glob(os.path.join(path, f"*.{ext}"))
    if not files:
        return None
    return max(files, key=os.path.getctime)


# ============================================================
# 06 - Flujo de navegación EUROPA
# ============================================================
def login_and_open_csps_europe():
    """Login en portal CNH y abrir CSPS EUROPA en nueva pestaña."""
    driver.get(LOGIN_URL)
    safe_find(By.ID, "userID", 20).send_keys(USER)
    driver.find_element(By.ID, "password").send_keys(PWD)
    driver.find_element(By.ID, "btn-submit").click()

    # Esperar que pase a my.dlrportal.com
    wait.until(EC.url_contains("my.dlrportal.com"))

    # Abrir CSPS EUROPA en nueva pestaña
    driver.switch_to.new_window("tab")
    driver.get(CSPS_URL)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "frameset")))


def open_consulta_multiple_stock_europe():
    """Menú Europa -> Consulta de Referencias -> Consulta Múltiple de Stock."""
    back_to_default()
    switch_to_frame("menu", 20)

    # 1) Expandir el folder "Consulta de Referencias"
    safe_click(By.ID, "folderIcon14", 20)
    time.sleep(1)

    # 2) Click en "Consulta Múltiple de Stock"
    link = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//a[@title='Consulta Múltiple de Stock']"))
    )
    driver.execute_script("arguments[0].click();", link)

    # 3) Cambiar al frame de detalle
    back_to_default()
    switch_to_frame("detail", 20)

    # Validar que está el input file
    safe_find(By.ID, "filePath", 20)


# ============================================================
# 07 - Main (ejecución principal)
# ============================================================
try:
    if not os.path.isdir(PARTS_DIR):
        print(f"[ERROR] No existe carpeta de .txt: {PARTS_DIR}")
        sys.exit(1)

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    login_and_open_csps_europe()
    open_consulta_multiple_stock_europe()

    for fname in sorted(os.listdir(PARTS_DIR)):
        if not fname.endswith(".txt"):
            continue

        file_path = os.path.join(PARTS_DIR, fname)
        print(f"[INFO] Subiendo {file_path}")

        # Subir archivo
        file_input = safe_find(By.ID, "filePath", 20)
        file_input.send_keys(file_path)

        # Ejecutar la función JavaScript fSearch() o el submit equivalente
        print("[ACTION] Ejecutando búsqueda...")
        driver.execute_script("""
            try {
                if (typeof fSearch === 'function') {
                    fSearch();
                } else {
                    document.forms['OpPa09pagMultiPartStockFormBean'].action =
                        '/orderprocessing/OpPa09pagMutiPartStockInqAction.do?submitDispatchAction=Search';
                    document.forms['OpPa09pagMultiPartStockFormBean'].submit();
                }
            } catch(e) {
                console.log('Error ejecutando fSearch():', e);
                document.forms['OpPa09pagMultiPartStockFormBean'].action =
                    '/orderprocessing/OpPa09pagMutiPartStockInqAction.do?submitDispatchAction=Search';
                document.forms['OpPa09pagMultiPartStockFormBean'].submit();
            }
        """)

        # Esperar resultados
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//td[contains(., 'Referencia')]")
        ))
        print(f"[OK] Resultados cargados para {fname}")
        time.sleep(2)

        # Descargar Excel
        safe_click(By.NAME, "Download", 20)
        print(f"[OK] Descargando resultado de {fname}")

        # Renombrar archivo descargado
        time.sleep(6)
        latest = get_latest_download()
        if latest:
            new_name = os.path.splitext(fname)[0] + ".xls"
            new_path = os.path.join(OUTPUT_DIR, new_name)
            try:
                shutil.move(latest, new_path)
                print(f"[OK] Guardado como {new_path}")
            except Exception as e:
                print(f"[WARN] No se pudo renombrar {latest}: {e}")

    print("[FIN] Todas las partes procesadas.")

except Exception as e:
    print("[ERROR]", repr(e))
finally:
    try:
        input("ENTER para cerrar...")
    except:
        pass
    driver.quit()


[INFO] Subiendo C:\partes_txt\part1.txt
[ACTION] Ejecutando búsqueda...
[OK] Resultados cargados para part1.txt
[OK] Descargando resultado de part1.txt
[OK] Guardado como C:\descargas\EURO\part1.xls
[INFO] Subiendo C:\partes_txt\part10.txt
[ACTION] Ejecutando búsqueda...
[OK] Resultados cargados para part10.txt
[OK] Descargando resultado de part10.txt
[OK] Guardado como C:\descargas\EURO\part10.xls
[INFO] Subiendo C:\partes_txt\part11.txt
[ACTION] Ejecutando búsqueda...
[OK] Resultados cargados para part11.txt
[OK] Descargando resultado de part11.txt
[OK] Guardado como C:\descargas\EURO\part11.xls
[INFO] Subiendo C:\partes_txt\part12.txt
[ACTION] Ejecutando búsqueda...
[OK] Resultados cargados para part12.txt
[OK] Descargando resultado de part12.txt
[OK] Guardado como C:\descargas\EURO\part12.xls
[INFO] Subiendo C:\partes_txt\part13.txt
[ACTION] Ejecutando búsqueda...
[OK] Resultados cargados para part13.txt
[OK] Descargando resultado de part13.txt
[OK] Guardado como C:\descargas\EURO\

In [2]:
import os
import xlrd
from openpyxl import Workbook

# Ruta de la carpeta de descargas
carpeta = r"C:\descargas\EURO"
archivos = [f for f in os.listdir(carpeta) if f.endswith(".xls")]

# Crear un archivo Excel nuevo
wb = Workbook()
ws = wb.active
ws.title = "Combinado"

fila_actual = 1  # para llevar control de en qué fila vamos

for archivo in archivos:
    ruta = os.path.join(carpeta, archivo)
    print(f"Procesando: {archivo}")

    # Abrir .xls con xlrd
    libro = xlrd.open_workbook(ruta)
    hoja = libro.sheet_by_index(0)  # primera hoja

    # Recorrer filas y celdas
    for r in range(hoja.nrows):
        valores = []
        for c in range(hoja.ncols):
            valores.append(hoja.cell_value(r, c))

        # Escribir en el Excel final
        for col, valor in enumerate(valores, start=1):
            ws.cell(row=fila_actual, column=col, value=valor)

        fila_actual += 1  # siguiente fila

# Guardar archivo combinado
salida = os.path.join(carpeta, "Combinado.xlsx")
wb.save(salida)
print(f"✅ Todos los .xls combinados en: {salida}")

Procesando: part1.xls
Procesando: part10.xls
Procesando: part11.xls
Procesando: part12.xls
Procesando: part13.xls
Procesando: part14.xls
Procesando: part15.xls
Procesando: part16.xls
Procesando: part17.xls
Procesando: part18.xls
Procesando: part19.xls
Procesando: part2.xls
Procesando: part20.xls
Procesando: part21.xls
Procesando: part22.xls
Procesando: part23.xls
Procesando: part24.xls
Procesando: part25.xls
Procesando: part26.xls
Procesando: part27.xls
Procesando: part28.xls
Procesando: part29.xls
Procesando: part3.xls
Procesando: part30.xls
Procesando: part31.xls
Procesando: part32.xls
Procesando: part33.xls
Procesando: part34.xls
Procesando: part35.xls
Procesando: part36.xls
Procesando: part37.xls
Procesando: part38.xls
Procesando: part39.xls
Procesando: part4.xls
Procesando: part40.xls
Procesando: part41.xls
Procesando: part42.xls
Procesando: part43.xls
Procesando: part44.xls
Procesando: part45.xls
Procesando: part46.xls
Procesando: part47.xls
Procesando: part48.xls
Procesando: par